# Sandbox
For testing and developing new Cyber Security Assessment tools in an interactive and persistent development environment.

In [ ]:
import itertools
import json
import copy
import random
import warnings
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats.distributions as distr
import seaborn as sns
import pandapower
from pathlib import Path as p

from cyber.assets import Defence, Vulnerability, CommmonDefences, CyberDevice
from communication.graph import CommNode, CommEdge
from communication.network import Aggregator, Device, CommNetwork
from attackers.random_attacker import RandomAttacker
from visualization import plot_communication_network

## Procedural Generation
### Abstract Tree
Consists of Devices and Aggregators. 
* Aggregators (internal nodes) require a **Hard** amount of effort to compromise and have a 50% chance of being compromised if the necessary effort is spent
* Devices (leaf nodes) require an **Easy** amount of effort to compromise and also have a 50% chance of being compromised if the necesssary effort is spent
* Control Center (root node) is **Very Hard** to compromise

Controllable parameters include:
* Number of devices (leaf nodes)
* Number of Entrypoints (points where cyberattacks can originate)
* Number of children per parent node (inversely proportional to redundancy)
* Random deviation in number of children
* Sibling to Sibling communication (lateral edges between nodes on the same level)

In [ ]:
seed = np.random.randint(low=0, high=52600)
np.random.seed(seed); random.seed(seed)
print(f"Seed: {np.random.get_state()[1][0]}")

with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid=pandapower.networks.case14()
    print(grid)
    pcn = CommNetwork(n_devices=3, n_entrypoints=1, children_per_parent=0, child_no_deviation=5, 
                    network_specs=p.cwd() / "specifications" / "SmartMeter_specifications.json", 
                    # "Default_specifications.json", "SmartMeter_specifications.json", "SCADA_specifications.json", "WAMS_specifications.json"
                    grid=None, # grid,
                    enable_sibling_to_sibling_comm=True)
print(CommNetwork.show_tree(pcn.root))
print(f"Number of Components: {pcn.n_components}")

## Visualization
Plot the structure of the communication network. 

In [ ]:
plot_communication_network(pcn, palette="tab10") # "tab10" (default), "Set2", "Paired", "flare"

## Monte Carlo
Build an approximate profile of the network's cyber security by launching many cyber attacks. The higher N_ATTACKS the more precise the resulting distribution is, however this comes at the cost of increased computation time.
The more nodes are compromised, the more successful the attack.

### Active Graph Only
Only perform Monte Carlo simulation on the currently active network.

In [ ]:
N_ATTACKS = 1000
BUDGET = 52
compromised_array = np.zeros(shape=N_ATTACKS, dtype=np.int16)
effort_array = np.zeros(shape=N_ATTACKS, dtype=np.float32)
for attack_no in range(N_ATTACKS):
    attacker = RandomAttacker(budget=BUDGET, verbose=False)
    nodes_compromised, total_effort_spent = attacker.attack_network(pcn)
    compromised_array[attack_no] = len([n for n in nodes_compromised if isinstance(n, Device)])
    effort_array[attack_no] = total_effort_spent
    pcn.reset()

In [ ]:
# Histogram
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8,6))
fig.suptitle(f"Attacker: {attacker.__name__}, Budget: {BUDGET}\nNetwork Size: {pcn.n_components}, No. of Devices: {pcn.n_devices}, No. of Entrypoints: {pcn.n_entrypoints}", 
             y=-0.05, fontsize="medium", ma="center")
sns.histplot(compromised_array, discrete=True, stat="probability", ax=axes[0])
axes[0].set(xticks=np.arange(0, len(pcn.graph.nodes())), xlabel="No. of Devices Compromised")
sns.histplot(effort_array, binwidth=1, ax=axes[1])
axes[1].set(xlabel="Effort Spent")
plt.tight_layout()
plt.show()

### Varied Parameter
Perform monte carlo simulation while varying particular parameter, such as the level of redundancy in the network. 

In [ ]:
import os
import multiprocess as mp
N_ATTACKS = 10000
N_DEVICES = 30
BUDGET = 52
SPEC = p.cwd() / "SmartMeterNetworkSpecifications.json"
SEED = np.random.randint(low=0, high=52600)
N_ENTRYPOINTS = 1 # Total budget is multiplied by this!
MIN_CHILDREN = 2
MAX_CHILDREN = N_DEVICES
CHILD_NO_STEP = 2
CHILD_NO_DEVIATION = 0
no_of_children = np.arange(MIN_CHILDREN, MAX_CHILDREN, CHILD_NO_STEP)
network_specs = dict(n_devices=N_DEVICES,
                     n_entrypoints=N_ENTRYPOINTS,
                     network_specs=SPEC,
                     child_no_deviation=CHILD_NO_DEVIATION,
                     enable_sibling_to_sibling_comm=True)

print(f"Seed: {SEED}")
np.random.seed(SEED)

compromised_array = np.zeros(shape=(N_ATTACKS, len(no_of_children)), dtype=np.int16)
effort_array = np.zeros(shape=(N_ATTACKS, len(no_of_children)), dtype=np.float32)
print(f"CPU Thread Count: {mp.cpu_count()-2}")

def monte_carlo(process_idx, seed, n_attacks, budget, **network_kwargs):
    import os
    import numpy as np
    from comm_network import CommNetwork, Device
    from attackers import RandomAttacker
    
    # Procedurally generate a communication network with specific redundancy
    np.random.seed(seed)
    pcn = CommNetwork(**network_kwargs)

    # Store effort and no. of devices compromised
    compromised_array = np.zeros(shape=n_attacks, dtype=np.int16)
    effort_array = np.zeros(shape=n_attacks, dtype=np.float32)

    for attack_no in range(n_attacks):
        attacker = RandomAttacker(budget=budget, verbose=False)
        nodes_compromised, total_effort_spent = attacker.attack_network(pcn)
        compromised_array[attack_no] = len([n for n in nodes_compromised if isinstance(n, Device)])
        effort_array[attack_no] = total_effort_spent
        # Entrypoint changes with each attack (i.e. same network different entrypoint)
        pcn.reset() 
    return process_idx, compromised_array, effort_array

with mp.Pool(processes=len(no_of_children)) as pool:
    results = []
    for i, children_per_parent in enumerate(no_of_children):
        print("Children per parent:", children_per_parent)
        kwds = {**network_specs, **dict(children_per_parent=children_per_parent)}
        results.append(
            pool.apply_async(monte_carlo, args=[i, SEED, N_ATTACKS, BUDGET], kwds=kwds)
        )
    
    for result in results:
        process_idx, compromises, efforts = result.get()
        compromised_array[:, process_idx] = compromises
        effort_array[:, process_idx] = efforts

In [ ]:
import pandas as pd
df = pd.DataFrame(compromised_array, columns=no_of_children)
df = df.melt(var_name='Children')

display(df)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
sns.histplot(df, x="value", hue="Children", discrete=True, stat="probability", common_norm=False, ax=ax)
sns.move_legend(ax, "upper right", ncols=4, title="Children per Parent")
ax.set(xlabel="No. of Devices Compromised")
plt.show()

## Static Analysis
Given an infinite budget, breaksdown the probability of compromising components in the network. The resulting probabilities are exact (except for floating point precision issues) but do not scale well to larger communication networks (> 5 nodes). Useful as a static feature of a communication network. 

In [ ]:
from cyber.analysis import static_analysis

n_probs = static_analysis(pcn, show_paths=False, verbose=True)

In [ ]:
# Adjacency Matrix
# Does not handle self-loops / backtracking
# Consequently, probabilities will differ from combinatorial approach

def superscript(num:int):
    sup_map = {0: f"\N{SUPERSCRIPT ZERO}", 1: f"\N{SUPERSCRIPT ONE}", 2: f"\N{SUPERSCRIPT TWO}", 3: f"\N{SUPERSCRIPT THREE}", 4: f"\N{SUPERSCRIPT FOUR}", 
           5: f"\N{SUPERSCRIPT FIVE}",  6: f"\N{SUPERSCRIPT SIX}", 7: f"\N{SUPERSCRIPT SEVEN}", 8: f"\N{SUPERSCRIPT EIGHT}", 9: f"\N{SUPERSCRIPT NINE}"}
    return "".join(sup_map[digit] for digit in map(int, str(num)))

np.set_printoptions(precision=2, floatmode="maxprec_equal")
nodes = sorted(pcn.graph.nodes(), key=lambda node: node.id)
probs = [node.get_prob_to_compromise() for node in nodes]
print(f"Probabilities: {probs}")


A = nx.adjacency_matrix(pcn.graph, nodelist=nodes, weight="p").todense()
n_probs = {}
oldA = np.eye(A.shape[0])
for i in range(len(nodes)):
    newA = oldA@A
    # np.fill_diagonal(newA, val=0)
    print(f"A{superscript(i+1)}\n", newA)
    n_probs[i+1] = np.triu(newA, k=1).sum()
    print(f"A{superscript(i+1)}: {n_probs[i+1]}")
    oldA = newA

In [ ]:
# Mutually Exclusive Approach
# Assumes you can jump and independently attack any node (i.e. ignores communication connections!)

time_required = 0.0
nodes = pcn.graph.nodes()
node_probs = {node: node.get_prob_to_compromise() for node in nodes}

n_probs = {}
all_nodes = set(nodes)
cumulative = 0.0
for n_devices in range(pcn.n_components, 0, -1):
    n_probs[n_devices] = cumulative
    for combination in itertools.combinations(nodes, n_devices):
        probability_to_compromise = 1.0
        combination = set(combination)
        missing_nodes = all_nodes.difference(combination)
        for node in combination:
            probability_to_compromise *= node_probs[node]
        for node in missing_nodes:
            probability_to_compromise *= (1 - node_probs[node])
        n_probs[n_devices] += probability_to_compromise 
    print(f"{n_devices} Devices: {n_probs[n_devices]}")
    cumulative += n_probs[n_devices]
print("\n".join(f"{k} devices: {v}" for k,v in sorted(n_probs.items(),key=lambda item: item[0])))
print("Sum:", sum(n_probs.values()))

In [ ]:
# If the probability of compromising all components is the same,
# we can use the Binomial distribution function
# Takes: 12.6 µs
N = pcn.n_components
k = 2
p = 0.5
cumulative = 0.0
for k in range(N, 0, -1):
    prob = math.comb(N, k)*math.pow(p, k)*math.pow(1-p,N-k)
    print(f"{k} Devices: {cumulative + prob}")
    cumulative += prob

In [ ]:
import scipy.stats.distributions as distr
distr_lookup = {
    "TruncNorm": distr.truncnorm, # Continuous, loc=mean (float), scale=standard deviation (float)
    "Exponential": distr.expon, # Continuous, scale = 1 / lambda (float)
    "Gamma": distr.gamma, # Continuous, a = shape parameter (integer)
    "Bernoulli": distr.bernoulli, # Discrete
}
n_attacks = 20
is_successful = distr.bernoulli(0.5).rvs(size=n_attacks).astype(bool)
time_taken = distr.expon(scale=0.0).rvs(size=n_attacks)[is_successful]
print(f"Successful Attacks {sum(is_successful)}/{n_attacks}\nTime Taken per Successful Attack: {time_taken}")

## Communication Network Specifications
Explores how we can supply structured information to our procedural network generation algorithm. Includes information such as the types of components and defences we expect to see in the communication network.

In [ ]:
# seed = np.random.randint(low=0, high=52600)
seed = 27194
print(f"Seed: {seed}")
np.random.seed(seed)
pcn = CommNetwork(n_devices=15, n_entrypoints=1, children_per_parent=5, child_no_deviation=1,
                  network_specs="SmartMeterNetworkSpecifications.json",
                  enable_sibling_to_sibling_comm=True)


## Criticality

In [ ]:
import inspect
import warnings
import numpy as np
import pandapower as pp
import pandapower.networks as grids
grid = pp.create_empty_network()
grid_filter = lambda module: inspect.isfunction(module) and not module.__name__.startswith("_")
grid_map = {grid_name:grid_creator for grid_name, grid_creator in \
            inspect.getmembers(grids, predicate=grid_filter)}
grid_options = list(grid_map.keys())
print(", ".join(grid_map.keys()))
CHOSEN_GRID = "mv_oberrhein" # "create_cigre_network_mv" # Can be None
kwargs = dict(scenario="generation", include_substations=True) #  dict(with_der="all")
with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid_name = np.random.choice(grid_options) if CHOSEN_GRID is None else CHOSEN_GRID
    print(f"Grid: {grid_name}")
    grid = grid_map[grid_name](**kwargs)
    print(grid)
    # Controllable
    n_controllable = sum(getattr(grid, attr).shape[0] for attr in ["gen", "shunt", "trafo", "switch"])
    print(f"No. of controllable elements: {n_controllable} (generators, shunts, transformers and switches)")
    # Sensor-Only
    n_sensor_only = sum(getattr(grid, attr).shape[0] for attr in ["bus", "load", "line"])
    print(f"No. of sensor-only elements: {n_sensor_only} (buses, loads and lines)")
    print(f"Total (possible) no. of devices: {n_sensor_only+n_controllable} (generators, shunts, transformers and switches, buses, loads and lines)")

# grid.switch.closed = True
pp.plotting.simple_plot(grid, respect_switches=True, plot_line_switches=True, plot_loads=True, plot_gens=True, plot_sgens=True, )


In [ ]:
import pandas as pd

class CommNetwork(object):
    """
    Procedurally generated communication network composed of Devices and Aggregators.
    Each aggregator has a certain amount of children (can be devices, or other aggregators).
    Entry points are points in the network that can potentially be used by attackers to try
    and compromise the network, such as a Remote connection to a substation.
    """
    __name__ == "CommNetwork"

    def __init__(self, n_devices:int=20,
                 children_per_parent:int=3, child_no_deviation:int=2,
                 network_specs:p=p.cwd() / "specifications" / "Default_specifications.json",
                 grid:pandapower.pandapowerNet|grid2op.Environment.BaseEnv|None=None,
                 enable_sibling_to_sibling_comm:bool=False,
                 n_entrypoints:int=3):
        """
        The topology of the communication network is procedurally generated based on the
        parameters set here.

        Args:
            n_devices (int, optional):
                Number of end-devices which collect data or execute commands.
                Defaults to 20.
            children_per_parent (int, optional):
                Level of redundancy in the network, i..e the no. of children per aggregator.
                Defaults to 3.
            child_no_deviation (int, optional):
                Random variation in the redundancy, ignored if redundancy is NONE or FULL.
                Defaults to 2.
            network_specs (str, optional):
                Specifications of network. This is a JSON dictionary that provides details on
                devices, aggregators and the root node.
                Defaults to {}.
            grid (pandapower.pandapowerNet | grid2op.Environment.BaseEnv | None, optional):
                Specific pandapower grid to map communication network to. Will override 'n_devices'.
                Defaults to None.
            enable_sibling_to_sibling_comm (bool, optional):
                Whether to have lateral connections as well between nodes with the same parent.
                Defaults to False.
            n_entrypoints (int, optional):
                Number of entrypoints for attackers.
                Defaults to 3.
        """
        self.n_devices = n_devices

        # Redundancy (no. of children per aggregator)
        self.children_per_parent = children_per_parent
        self.child_no_deviation = child_no_deviation
        
        with open(network_specs, "r", encoding="utf-8") as f:
            self.specs = json.load(f, cls=SpecDecoder)
        self.grid = grid
        self.enable_sibling_to_sibling_comm = enable_sibling_to_sibling_comm
        self.n_entrypoints = n_entrypoints

        # Generate Communication Network (Procedurally)
        self.n_components = 0
        self.node_ids = []
        self.id_to_node = {} # Does not include root
        self.root = self.build_network(components=[])
        self.entrypoints = []
        self.set_entrypoints()
        self.graph = self.build_graph(self.root, nx.DiGraph())
        
    def build_leaves(self):
        """
        Construct the leaf nodes of the network. Leaf nodes have no children.

        Returns:
            list[TreeNode]: Collection of leaf nodes
        """
        components = []
        device_types = self.specs["device"]["types"]
        # Proportion of devices of each type (default: uniform)
        device_type_prob = self.specs["device"].get("proportion", [1/len(device_types)]*len(device_types))
        if self.grid is None: 
            # Device Type is based on statistic / expected proportion
            device_map = enumerate(np.random.choice(device_types, p=device_type_prob,
                                                    replace=True, size=self.n_devices))
        else:
            # Device Type is based on compatibility with power grid element in PandaPower
            compatabilities = {}
            for device_type in device_types:
                compatible_devices = device_type.get("compatible")
                for comp_device_type in compatible_devices.keys():
                    if comp_device_type not in compatabilities:
                        compatabilities[comp_device_type] = [device_type]
                    else:
                        compatabilities[comp_device_type] = compatabilities[comp_device_type] + [device_type]

            # Map device type (by name) to probability that device is of that type
            probs = {device_type.get("name"): prob for device_type, prob in zip(device_types, device_type_prob)}

            no_of_devices = 0
            device_map = {}
            if isinstance(self.grid, pandapower.pandapowerNet):
                # Allocation per element (ignored conditions)
                for attr, compatible_device_types in compatabilities.items():
                    attr_df = getattr(self.grid, attr)
                    obj_ids = CommNetwork.evaluate_pandapower_conditions(device_type, attr_df, attr)
                    no_of_attr_devices = len(obj_ids)
                    for i in range(no_of_devices, no_of_devices+no_of_attr_devices):
                        # Recalculate probability of choosing each compatible device
                        # Retains original proportion, probabilities must sum to 1
                        local_probs = np.array([probs[device_type.get("name")] for device_type in compatible_device_types])
                        local_probs = (1/sum(local_probs))*local_probs
                        device_map[i] = np.random.choice(compatible_device_types, p=local_probs)
                    no_of_devices += no_of_attr_devices
                device_map = device_map.items()
            elif isinstance(self.grid, grid2op.Environment.BaseEnv):
                # Allocation per substation (evaluates conditions)

                # Grid2Op Obj name mapping
                grid2op_naming = {"load": "loads_id", "gen":"generators_id", "line":"lines_or_id", "storage":"storages_id"}

                for sub_no, sub_name in enumerate(self.grid.name_sub):
                    connected_objs = self.grid.get_obj_connect_to(substation_id=sub_no)
                    
                    for attr, compatible_device_types in compatabilities.items():
                        obj_ids = connected_objs[grid2op_naming[attr]]
                        # Filter / Extend IDs depending on 'device_type' conditions
                        obj_ids = CommNetwork.evaluate_grid2op_conditions(device_type, self.grid.get_obs(), obj_ids, attr)
                        no_of_attr_devices =len(obj_ids)
                        for i in range(no_of_devices, no_of_devices+no_of_attr_devices):
                            # Recalculate probability of choosing each compatible device
                            # Retains original proportion, probabilities must sum to 1
                            local_probs = np.array([probs[device_type.get("name")] for device_type in compatible_device_types])
                            local_probs = (1/sum(local_probs))*local_probs
                            device_map[i] = np.random.choice(compatible_device_types, p=local_probs)
                        no_of_devices += no_of_attr_devices
                device_map = device_map.items()
                pass
            else:
                raise NotImplemented(f"Grid of type ({type(self.grid)}) is not yet supported.")
        
        # Create Devices
        for i, device_type in device_map:
            device_attrs =  CommNetwork.get_binary_attributes(device_type,
                            ["is_sensor", "is_controller", "is_accessible", "is_autonomous"])
            device = Device(name=device_type.get("name", "Device"),
                            is_controller=device_attrs["is_controller"],
                            is_sensor=device_attrs["is_sensor"],
                            is_autonomous=device_attrs["is_autonomous"],
                            is_accessible=device_attrs["is_accessible"],)
            CommNetwork.attach_cyber_characteristics(device, device_type)
            components.append(device)
            self.node_ids.append(device.id)
            self.id_to_node[device.id] = device
        return components
    
    def build_aggregators(self, components:list[TreeNode]):
        """
        Construct the aggregator nodes of the network. Each aggregator node oversees 1 or
        more components 1 level below it in the hierarchy. 

        Args:
            components (list[TreeNode]): Nodes 1 level lower in the hierarchy.

        Returns:
            list[TreeNode]: Collection of aggregator nodes
        """
        aggregators = []

        # Allocate children/components to each Aggregator
        children_per_aggregator = []
        skipped_children = []
        while (sum_so_far := sum(children_per_aggregator)) != len(components):
            # Allow some variation in no. of children
            if self.child_no_deviation > 0:
                random_deviation = np.random.randint(-self.child_no_deviation, self.child_no_deviation + 1)
            else:
                random_deviation = 0
            # No. of children per aggregator (can be negative)
            n_children = self.children_per_parent + random_deviation
            
            if n_children < 0: # Negative cannot exceed remaining no. of components
                n_children = max(n_children, sum_so_far - len(components))
            else: # Positive must at least be 1, up to the maximum no. of remaining components
                n_children = max(1, min(n_children, len(components) - sum_so_far))
            children_per_aggregator.append(np.abs(n_children)) 

            if n_children <= 1: # Assign children to higher level in hierarchy
                skipped_children.extend(components[sum_so_far:sum_so_far+np.abs(n_children)] if \
                                        np.abs(n_children) > 1 else [components[sum_so_far]])
            else: # Assign children to a new aggregator
                # Create the aggregator
                aggregator_type = np.random.choice(self.specs["aggregator"]["types"],
                                                   p=self.specs["aggregator"].get("proportion",None))
                aggregator_attrs =  CommNetwork.get_binary_attributes(aggregator_type, ["is_accessible"])
                aggregator = Aggregator(name=aggregator_type.get("name", "Aggregator"),
                                        is_accessible=aggregator_attrs["is_accessible"])
                CommNetwork.attach_cyber_characteristics(aggregator, aggregator_type)
            
                # Connects Edges
                for i, component in enumerate(components[sum_so_far:sum_so_far+n_children]):
                    component.update_parents(aggregator)
                    CommNetwork.connect_by_edges(aggregator, component)
                    # Connect siblings
                    if i >= 1 and self.enable_sibling_to_sibling_comm:
                        prev_component = components[sum_so_far + (i-1)]
                        if prev_component.__class__ == component.__class__:
                            CommNetwork.connect_by_edges(prev_component, component)

                # Keep Track of Nodes
                aggregators.append(aggregator)
                self.node_ids.append(aggregator.id)
                self.id_to_node[aggregator.id] = aggregator
        aggregators.extend(skipped_children)
        return aggregators
    
    def build_root(self, components:list[TreeNode]):
        """
        Construct the root node of the network.

        Args:
            components (list[TreeNode]): Nodes one level below the root in the hierarchy.

        Returns:
            TreeNode: Root of the communication network
        """
        # Create the root node
        root_type = self.specs["root"]
        root_attrs =  CommNetwork.get_binary_attributes(root_type, ["is_accessible"])
        root = Aggregator(name=root_type.get("name", "Control Center"),
                          is_accessible=root_attrs["is_accessible"])
        CommNetwork.attach_cyber_characteristics(root, root_type)
        
        for i, component in enumerate(components):
            component.update_parents(root)
            CommNetwork.connect_by_edges(root, component)
            # Connect siblings
            if i >= 1 and self.enable_sibling_to_sibling_comm:
                prev_component = components[i - 1]
                if prev_component.__class__ == component.__class__:
                    CommNetwork.connect_by_edges(prev_component, component)
        return root
        
    def build_network(self, components:list[Device|Aggregator]):
        """
        Recursively build Communication Network composed of Devices and Aggregators.

        Args:
            components (list[Device|Aggregator]): Collection of components (Device or Aggregator) at current level. Defaults to [].

        Returns:
            Aggregator: Root node of tree (represents control center)
        """
        if len(components) == 0:
            components = self.build_leaves()
            self.n_components += len(components)
        elif len(components) > 1:
            components = self.build_aggregators(components)
            self.n_components += len(components)
        else:
            root = self.build_root(components)
            self.n_components += 1
            return root
        return self.build_network(components)
    
    def set_entrypoints(self):
        """
        Randomly set entry points at devices or aggregators in the network.
        Excludes control center / root.
        """
        # Reset any existing entrypoints
        for entrypoint in self.entrypoints:
            entrypoint.is_accessible = False
        self.entrypoints = []
        accessible_ids = np.random.choice(self.node_ids,
                                              min(self.n_components - 1, self.n_entrypoints),
                                              replace=False)
        for accessible_id in accessible_ids:
            component = self.id_to_node[accessible_id]
            component.is_accessible = True
            self.entrypoints.append(component)
        
        # self.walk_and_set_entrypoints(self.root, ids_to_match=accessible_components)
    
    def build_graph(self, root:Aggregator, graph:nx.DiGraph):
        """
        Construct NetworkX Graph from connected Aggregators / Devices

        Args:
            root (Aggregator): Root node of the Communication Network (e.g. the Control Center)
        Returns:
            networkx.DiGraph: Directional NetworkX Graph, with added nodes/edges
        """
        graph.add_node(root)
        for edge in root.outgoing_edges:
            graph.add_edge(edge.source, edge.target)
        for edge in root.incoming_edges:
            graph.add_edge(edge.target, edge.source)
        for child in root.children:
            graph = self.build_graph(child, graph)
        return graph
    
    def walk_and_set_entrypoints(self, root:Aggregator, ids_to_match:np.ndarray):
        """
        Walk the tree, modifying indices that are present in the 'idcs_to_match' array.

        Args:
            root (Aggregator): _description_
            attr_name (str): Name of attribute to modify
            set_value (object): Value to set the attribute to set
            idcs_to_match (np.ndarray): Idcs (in walking order) to modify
            idx (int, optional): Current index in walk. Defaults to 0.

        Returns:
            int: Last visited index
        """
        if root.id in ids_to_match:
            root.is_accessible = True
            self.entrypoints.append(root)
        for child in root.children:
            self.walk_and_set_entrypoints(child, ids_to_match)
    
    def reset(self):
        """
        Resets the network, including setting new entrypoint(s)
        """
        self.set_entrypoints()
        self.reset_cyber_components(active_node=self.root)
        self.graph = self.build_graph(self.root, graph=nx.DiGraph())

    def reset_cyber_components(self, active_node=None):
        """
        Recursively reset the cyber security status of all components in
        the network, starts from the root node.
        """
        if active_node is None:
            active_node = self.root
        active_node.reset()
        for child in active_node.children:
            self.reset_cyber_components(active_node=child)
    
    @staticmethod
    def evaluate_grid2op_conditions(device_type:dict, obs:grid2op.Observation, obj_ids:np.ndarray, attr:str):
        """
        Checks each object ID at a specific substation to see if it meets the conditions
        given in the network's JSON specifications. 

        Returns:
            list: IDs that satisfy the conditions.
        """
        conditions = device_type.get("compatible", None).get(attr, None)
        device_ids = copy.deepcopy(obj_ids)
        if conditions is not None and len(obj_ids) > 0:
            for condition in conditions:
                values = getattr(obs, condition["attribute"])[list(set(device_ids))]
                match condition["action"]:
                    case "filter":
                        matching_ids = np.where((values >= condition.get("lb", -math.inf)) & \
                                                (values < condition.get("ub", math.inf)))
                        device_ids = device_ids[matching_ids]
                    case "split":
                        limit = condition.get("limit", math.inf)
                        ids_to_split = device_ids[np.where(values > limit)]
                        
                        new_ids = [obj_id for obj_id in device_ids if obj_id not in ids_to_split]
                        for i, id_to_split in enumerate(ids_to_split):
                            new_ids.extend([id_to_split]*math.ceil(values[i] / limit))
                        device_ids = new_ids
        return device_ids
    
    @staticmethod
    def evaluate_pandapower_conditions(device_type:dict, attr_df:pd.DataFrame, attr:str):
        """
        Checks each PandaPower element to see if it meets the conditions
        given in the network's JSON specifications. 

        Returns:
            list: Elements that satisfy the conditions.
        """
        conditions = device_type.get("compatible", None).get(attr, None)
        device_ids = copy.deepcopy(attr_df.index.values)
        if conditions is not None and len(df) > 0:
            for condition in conditions:
                values = attr_df.get(condition["attribute"], [])[list(set(device_ids))]
                match condition["action"]:
                    case "filter":
                        matching_ids = np.where((values >= condition.get("lb", -math.inf)) & \
                                                (values < condition.get("ub", math.inf)))[0]
                        device_ids = device_ids[matching_ids]
                    case "split":
                        limit = condition.get("limit", math.inf)
                        ids_to_split = device_ids[np.where(values > limit)]
                        
                        new_ids = [obj_id for obj_id in device_ids if obj_id not in ids_to_split]
                        for i, id_to_split in enumerate(ids_to_split):
                            new_ids.extend([id_to_split]*math.ceil(values[i] / limit))
                        device_ids = new_ids
        return device_ids


    @staticmethod
    def get_binary_attributes(configuration:dict, attributes:list[str], default:bool=False):
        """
        Retrieve named (binary/boolean) attributes from a configuration dictionary for a
        specific component type.

        Args:
            configuration (dict): Describes the attributes of a component
            attributes (list[str]): Names of attributes to try and retrieve
            default (bool, optional): Default value to use if attribute cannot be found.
                Defaults to False.

        Returns:
            dict[str:bool]: _description_
        """
        attrs = {}
        for attr_name in attributes:
            attr = configuration.get(attr_name, default)
            attrs[attr_name] = attr if type(attr) is bool else bool(attr.rvs())
        return attrs

    @staticmethod
    def attach_cyber_characteristics(component:CyberComponent, configuration:dict):
        """
        Add all Defences and Vulnerabilities specific in a component's configuration
        dictionary to that component.

        Args:
            component (CyberComponent): A component in the communication network.
            configuration (dict): Describes the characteristics of this type of
                component.
        """
        for defence in configuration.get("defences", []):
            component.add_defence(
                Defence(name=defence.get("name", "Defence"),
                        success_distribution=defence["success"],
                        effort_distribution=defence["effort"])
            )
        for vulnerability in configuration.get("vulnerabilities", []):
            component.attach_vulnerability(
                Vulnerability(name=vulnerability.get("name", "Vulnerability"))
            )

    @staticmethod
    def connect_by_edges(source:Device|Aggregator, target:Device|Aggregator):
        """
        Adds two-way communication edges depending on whether child is a sensor and/or controller or aggregator.
        TODO: One-way communication

        Args:
            source (Device|Aggregator): Component to connect from
            target (Device|Aggregator): Component to connect to
        """
        source.add_incoming_edge(target, Link(None, None))
        target.add_outgoing_edge(source, Link(None, None))

    @staticmethod
    def show_tree(root:Aggregator, s:str="", depth:int=0):
        """
        Recursively prints out structure of communication network using whitespace to denote deeper components.

        Args:
            root (Aggregator): _description_
            s (str, optional): _description_. Defaults to "".
            depth (int, optional): _description_. Defaults to 0.
            include_hash (bool): Whether to include the hash ID of each node

        Returns:
            str: String representing the network architecture
        """
        s += f"{depth*'   '}{root}\n"
        for child in root.children:
            s = CommNetwork.show_tree(child, s=s, depth=depth+1)
        return s

In [ ]:
seed = np.random.randint(low=0, high=52600)
np.random.seed(seed); random.seed(seed)
print(f"Seed: {np.random.get_state()[1][0]}")

with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid = pandapower.networks.mv_oberrhein(scenario="generation") # pandapower.networks.case14()
    print(grid)
    pcn = CommNetwork(n_devices=30, n_entrypoints=1, children_per_parent=0, child_no_deviation=5, 
                    network_specs=p.cwd() / "specifications" / "SCADA_specifications.json", 
                    # "Default_specifications.json", "SmartMeter_specifications.json", "SCADA_specifications.json", "WAMS_specifications.json"
                    grid=grid,
                    enable_sibling_to_sibling_comm=True)
print(CommNetwork.show_tree(pcn.root))
print(f"Number of Components: {pcn.n_components}")